# Código para aplicação das LB direto no dado para rotulação

## Preparando o test da base ouro para aplicar as labels functions

In [190]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [191]:
#apenas para testar o iob com spacy
#cria um novo dataset com as 2000 primeiras linhas
#csv_reader = pd.read_csv("/home/lucelia_vieira/Experimentos/ner/CSVs/REL_EXTRATO_CONTRATO_COMPLETO.csv", nrows=200)

In [192]:
#df = ["EXTRATO DO CONTRATO No 11/2019 PROCESSO: 00390-00003006/2019-92. ESPECIE: Contrato de Aquisicao de Bens pelo Distrito Federal no 11/2019. PARTES: Secretaria de Estado de Desenvolvimento Urbano e Habitacao do Distrito Federal - SEDUH/DF e INDUSTRIA DE AGUA MINERAL IBIA LTDA. DO OBJETO: Aquisicao de agua potavel de mesa sem gas e garrafao retornavel, destinados ao atendimento desta pasta, conforme demais especiGcacoes e condicoes estabelecidas no Termo de Referencia Anexo I do Edital de Pregao Eletronico no 0058/2018 - SCG/SEPLAG, da Proposta da empresa INDUSTRIA DE AGUA MINERAL IBIA LTDA de 11/06/2018 e da Lei no 8.666, de 21 de junho de 1993, que passam a integrar o presente Termo. DO VALOR: R$ 11.661,00 (onze mil, seiscentos e sessenta e um reais) conforme Nota de Empenho no 2019NE00180, emitida em 18/07/2019. Dotacao Orcamentaria P.T 15.122.6001.8517.0131 - Natureza da Despesa: 3390.30 - Fonte de Recursos: 100. DO PRAZO DE VIGENCIA: O Contrato tera vigencia de 06 (seis) meses a contar do dia 07/08/2019. DATA DE ASSINATURA: 22/07/2019. SIGNATARIOS: Pelo Distrito Federal: Mateus Leandro de Oliveira, na qualidade de Secretario de Estado. Pela Contratada: Eduardo Barros de Queiroz Rodrigues, na qualidade de Socio Administrador.", "EXTRATO DO CONTRATO Nº 06/2021 Processo: 00139-00001275/2021-00. Partes: DF - ADMINISTRAÇÃO REGIONAL DO CRUZEIRO X COMERCIAL DE ALIMENTOS PARK SUL. Objeto: O presente contrato tem por objeto a contratação de empresa especializada na área de alimentação para confecção e fornecimento de 50 KG DE BOLO CONFEITADO TIPO ANIVERSÁRIO COM MASSA PÃO DE LÓ COM RECHEIO DE BABA DE MOÇA, BEIJINHO OU DOCE DE LEITE E COBERTURA DE CHANTILLY DECORADO COM ACABAMENTO, no evento de comemoração ao aniversário de 62 anos do Cruzeiro no dia 30/11/2021. Valor do Contrato: R$ 2.000,00 (Dois Mil Reais). Dotação orçamentária: Unidade Orçamentária: 190113. Programa de Trabalho: 13.392.6219.3678.0136 . Natureza da Despesa: 339039. Fonte de Recursos: 120. Nota de Empenho: 2021NE00259. Valor do Empenho Inicial: R$ 2.000,00 (Dois Mil Reais), emitida em 26/11/2021 sob o evento 400091, na Modalidade 1   Ordinário. Fundamento Legal: Art. 24 da Lei nº 8.666/1993. Data da Assinatura: 29/11/2021. Signatários: Pelo DISTRITO FEDERAL: LUIZ EDUARDO GOMES DE PAULA PESSOA, na qualidade de Administrador Regional do Cruzeiro; pela COMERCIAL DE ALIMENTOS PARK SUL: GIVANILDES DE AGUIAR MORAIS, na qualidade de Gerente Geral."] 

In [193]:
#Carrega o dataset de teste para aplicacao das LB
#data = pd.read_csv('./CSVs/V1/Suspensao_Licitacao_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V1/Aditamento_Contrato_Split_0_x_test_texto.csv', dtype=str)
#data = pd.read_csv('./CSVs/V1/Anul_Revog_Licitacao_Split_0_x_test_texto.csv', dtype=str)
data = pd.read_csv('./CSVs/V1/Aviso_Licitacao_Split_0_x_test_texto.csv', dtype=str)

data

,Unnamed: 0,Texto
0,0,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 56/202...
1,1,AVISO DE LICITAÇÃO Procedimento Licitatório El...
2,2,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO SRP Nº 01...
3,3,AVISO DE ABERTURA DE LICITAÇÃO SISTEMA DE REGI...
4,4,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 53/202...
...,...,...
123,123,AVISO DE RETIFICAÇÃO ( * ) CONCORRÊNCIA Nº 003...
124,124,AVISO CONCORRÊNCIA Nº 02/2022 Processo : 00113...
125,125,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 07/202...
126,126,AVISO DE LICITAÇÃO - CONCORRÊNCIA Nº 03/2022 A...


In [194]:
#criando novo dataframe somente com os textos
df1 = data['Texto'].reset_index(drop=True)
#df1 = data['texto'].reset_index(drop=True)
df1

0      AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 56/202...
1      AVISO DE LICITAÇÃO Procedimento Licitatório El...
2      AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO SRP Nº 01...
3      AVISO DE ABERTURA DE LICITAÇÃO SISTEMA DE REGI...
4      AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 53/202...
                             ...                        
123    AVISO DE RETIFICAÇÃO ( * ) CONCORRÊNCIA Nº 003...
124    AVISO CONCORRÊNCIA Nº 02/2022 Processo : 00113...
125    AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 07/202...
126    AVISO DE LICITAÇÃO - CONCORRÊNCIA Nº 03/2022 A...
127    AVISO DE RETOMADA DA LICITAÇÃO Concorrência nº...
Name: Texto, Length: 128, dtype: object

In [195]:
#len(df_sk)

In [196]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA
#Para rodar extrato de contrato descomente abaixo as duas linhas
#import dodfSkweak_v1_2
#skw = dodfSkweak_v1_2.SkweakContratos(df1)

#Para rodar aditamento de contrato descomente abaixo as duas linhas
#import dodfSkweak_Aditamento
#skw = dodfSkweak_Aditamento.SkweakAditamento(df1)

#Para rodar Suspensao descomente abaixo as duas linhas
#import dodfSkweak_Suspensao
#skw = dodfSkweak_Suspensao.SkweakSuspensao(df1)


#Para rodar Anulacao e Revogação
#import dodfSkweak_Anul_Revog
#skw = dodfSkweak_Anul_Revog.SkweakAnulRevog(df1)

#Para rodar Aviso Licitação
import dodfSkweak_Aviso_Licitacao
skw = dodfSkweak_Aviso_Licitacao.SkweakAvisoLicitacao(df1)

skw.apply_label_functions()
skw.train_HMM_Dodf()

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


Starting iteration 1
Finished E-step with 128 documents
Starting iteration 2


         1       -3312.9298             +nan


Finished E-step with 128 documents
Starting iteration 3


         2       -3144.6159        +168.3139


Finished E-step with 128 documents
Starting iteration 4


         3       -3142.5704          +2.0455


Finished E-step with 128 documents


         4       -3142.0425          +0.5279


Write to ./data/aviso_licitacao.spacy...done


In [197]:
skw.list_spans_specific(0)

[PREGÃO ELETRÔNICO, 56/2021, 0011300005906/ 2021-95, 26.833,33]


In [198]:
#skw.list_spans_all()

In [199]:
data_label=skw.get_hmm_dataframe()
data_label

,processo_gdf,modalidade_licitacao,numero_licitacao,valor_estimado_contratacao,text,labels
0,0011300005906/ 2021-95,PREGÃO ELETRÔNICO,56/2021,"26.833,33",AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 56/202...,O O O B-modalidade_licitacao I-modalidade_lici...
1,00112-00012660/ 2021-36,,,"511.074,77",AVISO DE LICITAÇÃO Procedimento Licitatório El...,O O O O O O O O O O O O O O O O O O O O O O O ...
2,00390-00000127/ 2020-16,PREGÃO ELETRÔNICO,01/2021,"187.916,36",AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO SRP Nº 01...,O O O B-modalidade_licitacao I-modalidade_lici...
3,00401-00003779/ 2021-44,PREGÃO ELETRÔNICO,08/2021,,AVISO DE ABERTURA DE LICITAÇÃO SISTEMA DE REGI...,O O O O O O O O O O B-modalidade_licitacao I-m...
4,0004000033884/ 2020-91,PREGÃO ELETRÔNICO,53/2021,"33.140.640,08",AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 53/202...,O O O B-modalidade_licitacao I-modalidade_lici...
...,...,...,...,...,...,...
123,00113-00002946/ 2022-66,,003/2022,"5.257.461,51",AVISO DE RETIFICAÇÃO ( * ) CONCORRÊNCIA Nº 003...,O O O O O O O O B-numero_licitacao O O B-proce...
124,00113-00003425/ 2021-45,,002/2022,"5.899.518,69",AVISO CONCORRÊNCIA Nº 02/2022 Processo : 00113...,O O O B-numero_licitacao O O B-processo_gdf I-...
125,24,PREGÃO ELETRÔNICO,07/2022,"70.000,00",AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 07/202...,O O O B-modalidade_licitacao I-modalidade_lici...
126,00090-00001679/ 2022-42,,03/2022,,AVISO DE LICITAÇÃO - CONCORRÊNCIA Nº 03/2022 A...,O O O O O O B-numero_licitacao O O O O O O O O...


In [200]:
# converting to CSV file
#data_label.to_csv("./CSVs/ANUL_REVOG_SUPERVISÃO_FRACA_TESTE.csv")

In [201]:
corpus = skw.get_IOB()

In [202]:
len(corpus)

128

In [203]:
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y)


In [204]:
debug

{'B-modalidade_licitacao',
 'B-numero_licitacao',
 'B-processo_gdf',
 'B-valor_estimado_contratacao',
 'I-modalidade_licitacao',
 'I-processo_gdf',
 'O'}

In [205]:
len(termos_skweak)

128

In [206]:
termos_skweak_list = []
for i in termos_skweak:
   if not isinstance(i, list):
      termos_skweak_list .append(i)
   else:
      for j in i:
       termos_skweak_list .append(j)

len(termos_skweak_list )

21669

In [207]:
#termos_skweak_list

In [208]:
#apenas para conferencia
#df = pd.DataFrame(termos_skweak_list, columns=['Texto']) 
#df.to_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/termos_skweak_list.csv')

In [209]:
labels_skweak_list = []
for i in labels_skweak:
   if not isinstance(i, list):
      labels_skweak_list.append(i)
   else:
      for j in i:
        labels_skweak_list.append(j)

len(labels_skweak_list)

21669

## Preparando o test da base ouro para fazer a validação

In [210]:
#Carrega o dataset de teste da base ouro para aplicar a validação
#df_gold_ato= pd.read_csv('./CSVs/Extrato_Contrato_Split_1_x_test.csv', dtype=str)
#df_gold_ato

In [211]:
#Carrega os atos do dataset de teste da base ouro
#https://pt.stackoverflow.com/questions/232208/ler-uma-lista-de-listas-em-txt-python-3
import pickle
#Suspensao
#with open("./CSVs/V1/Suspensao_Licitacao_Split_0_x_test.txt", "rb") as file:
#Anul e Revog
#with open("./CSVs/V1/Anul_Revog_Licitacao_Split_0_x_test.txt", "rb") as file: 
#Aviso Licitacao
with open("./CSVs/V1/Aviso_Licitacao_Split_0_x_test.txt", "rb") as file: 
       
    x_test = pickle.load(file)

In [212]:
# Retira uma nível da lista dos atos do dataset de teste da base ouro
x_test_list = []
for i in x_test:
   if not isinstance(i, list):
      x_test_list.append(i)
   else:
      for j in i:
        x_test_list.append(j)

len(x_test_list)

21669

In [213]:
#Carrega os labels do dataset de teste da base ouro (Lista dentro de lista)
import pickle
#Suspensao
#with open("./CSVs/V1/Suspensao_Licitacao_Split_0_y_test.txt", "rb") as file:
#Anul e Revog
#with open("./CSVs/V1/Anul_Revog_Licitacao_Split_0_y_test.txt", "rb") as file:
#Aviso Licitacao
with open("./CSVs/V1/Aviso_Licitacao_Split_0_y_test.txt", "rb") as file:    
    y_test = pickle.load(file)

In [214]:
#  Retira uma nível da lista dos labels do dataset de teste da base ouro

y_test_list = []
for i in y_test:
   if not isinstance(i, list):
      y_test_list.append(i)
   else:
      for j in i:
       y_test_list.append(j)

len(y_test_list)

21669

In [215]:
len(termos_skweak_list)

21669

In [216]:
#Apenas checa se os tokens estão batento
resultados = []
for sk_ato , gold_ato in zip(termos_skweak_list, x_test_list):
    tupla=(sk_ato , gold_ato) 
    resultados.append(tupla)
    if sk_ato != gold_ato:
        print(sk_ato, gold_ato)

#for item in resultados:
#    print(item)


In [217]:
''' #labels_skweak = labels
numero_contrato = 0
numero_contrato_total = 0
B_numero_contrato = 0
numero_contrato_gold=[]
obj_sk=[]


data_assinatura_contrato = 0
data_assinatura_contrato_total = 0
B_data_assinatura_contrato = 0
data_assinatura_contrato_gold = []

unidade_orcamentaria = 0
unidade_orcamentaria_total = 0
B_unidade_orcamentaria = 0
unidade_orcamentaria_gold = []
teste=[]

programa_trabalho = 0
programa_trabalho_total = 0
B_programa_trabalho = 0
programa_trabalho_gold = []

for ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  gold_x_test_list, gold_y_test_list):    
    if label_gold == "B-numero_contrato":
           #print("Teste")
        numero_contrato_total+=1
        B_numero_contrato=label_gold
        if label_gold == label_sk:
               numero_contrato+=1
        else:
            numero_contrato_gold.append(ato_gold)

    elif label_gold == "B-data_assinatura_contrato":
                
        data_assinatura_contrato_total+=1
        B_data_assinatura_contrato=label_gold
        if label_gold == label_sk:
            data_assinatura_contrato+=1
        else:
            data_assinatura_contrato_gold.append(ato_gold)    
    elif label_gold == "B-unidade_orcamentaria":    
        unidade_orcamentaria_total+=1
        B_unidade_orcamentaria=label_gold
        if label_gold == label_sk:
            unidade_orcamentaria+=1        
        else:
            unidade_orcamentaria_gold.append(ato_gold) 
                 
    elif label_gold == "B-programa_trabalho":      
        programa_trabalho_total+=1
        B_programa_trabalho=label_gold
        if label_gold == label_sk:
            programa_trabalho+=1
        else:
            programa_trabalho_gold.append(ato_gold)                
print("Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak")                
print("B-numero_contrato",              "   ",numero_contrato_total,         "   ",numero_contrato)#,"      ","       ",numero_contrato_gold)
print("B-data_assinatura_contrato"," ",data_assinatura_contrato_total,"   ",data_assinatura_contrato)#,"      ",data_assinatura_contrato_gold)
print("B-unidade_orcamentaria",      "  ",unidade_orcamentaria_total,    "   ",unidade_orcamentaria)#,"          ",unidade_orcamentaria_gold)
print( "B-programa_trabalho",       "  ",programa_trabalho_total,       "   ",programa_trabalho)#,"             ",programa_trabalho_gold)

#print(programa_trabalho_gold)
#print(programa_trabalho_gold) '''

' #labels_skweak = labels\nnumero_contrato = 0\nnumero_contrato_total = 0\nB_numero_contrato = 0\nnumero_contrato_gold=[]\nobj_sk=[]\n\n\ndata_assinatura_contrato = 0\ndata_assinatura_contrato_total = 0\nB_data_assinatura_contrato = 0\ndata_assinatura_contrato_gold = []\n\nunidade_orcamentaria = 0\nunidade_orcamentaria_total = 0\nB_unidade_orcamentaria = 0\nunidade_orcamentaria_gold = []\nteste=[]\n\nprograma_trabalho = 0\nprograma_trabalho_total = 0\nB_programa_trabalho = 0\nprograma_trabalho_gold = []\n\nfor ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  gold_x_test_list, gold_y_test_list):    \n    if label_gold == "B-numero_contrato":\n           #print("Teste")\n        numero_contrato_total+=1\n        B_numero_contrato=label_gold\n        if label_gold == label_sk:\n               numero_contrato+=1\n        else:\n            numero_contrato_gold.append(ato_gold)\n\n    elif label_gold == "B-data_assinatura_contrato":\n                \n

In [218]:
#Compara os labels reais e os preditos para avalir a rotulação fraca

#labels_skweak = labels
numero_contrato = 0
numero_contrato_total = 0
B_numero_contrato = 0
numero_contrato_gold=[]

data_assinatura_contrato = 0
data_assinatura_contrato_total = 0
B_data_assinatura_contrato = 0
data_assinatura_contrato_gold = []

processo_gdf = 0
processo_gdf_total = 0
B_processo_gdf = 0
processo_gdf_gold = []

unidade_orcamentaria = 0
unidade_orcamentaria_total = 0
B_unidade_orcamentaria = 0
unidade_orcamentaria_gold = []
teste=[]

programa_trabalho = 0
programa_trabalho_total = 0
B_programa_trabalho = 0
programa_trabalho_gold = []

#for text_sk, label_skweak, text_gold,  label_gold in zip(termos_skweak, labels_skweak, atos, labels):
for ato_sk , label_sk, ato_gold, label_gold in zip(termos_skweak_list, labels_skweak_list,  x_test_list, y_test_list):  
    #for text_skw, termo_skweak, text_ato, termo_gold in zip(text_sk, label_skweak, text_gold,  label_gold ):
       
       #print("termo swkeak", termo_skweak) 
       if label_gold == "B-numero_contrato":
           #print("Teste")
           numero_contrato_total+=1
           B_numero_contrato=label_gold
           if label_gold == label_sk:
               numero_contrato+=1
           else:
               numero_contrato_gold.append(ato_gold)

       elif label_gold == "B-data_escrito":    
            data_assinatura_contrato_total+=1
            B_data_assinatura_contrato=label_gold
            if label_gold == label_sk:
                data_assinatura_contrato+=1
            else:
                data_assinatura_contrato_gold.append(ato_gold)
       
       elif label_gold == "B-processo_gdf":        
            processo_gdf_total+=1
            B_processo_gdf=label_gold
            if label_gold == label_sk:
                processo_gdf+=1
            else:
                processo_gdf_gold.append(ato_gold)         
                

       elif label_gold == "B-unidade_orcamentaria":     
            unidade_orcamentaria_total+=1
            B_unidade_orcamentaria=label_gold
            if label_gold == label_sk:
                unidade_orcamentaria+=1
                
            else:
                unidade_orcamentaria_gold.append(ato_gold) 
                 
       elif label_gold == "B-programa_trabalho":  
            programa_trabalho_total+=1
            B_programa_trabalho=label_gold
            if label_gold == label_sk:
                programa_trabalho+=1
            else:
                programa_trabalho_gold.append(ato_gold)                
print("Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak")                
print("B-numero_contrato",              "   ",numero_contrato_total,         "   ",numero_contrato,"      ","       ",numero_contrato_gold)
print("B-data_assinatura_contrato"," ",data_assinatura_contrato_total,"   ",data_assinatura_contrato,"      ",data_assinatura_contrato_gold)
print("B-processo_gdf"," ", processo_gdf_total,"   ",processo_gdf,"      ",processo_gdf_gold)

#print("B-unidade_orcamentaria",      "  ",unidade_orcamentaria_total,    "   ",unidade_orcamentaria)#,"          ",unidade_orcamentaria_gold)
#print( "B-programa_trabalho",       "  ",programa_trabalho_total,       "   ",programa_trabalho)#,"             ",programa_trabalho_gold)

#print(programa_trabalho_gold)

Tipo da entidade             Total Entidade Gold                 Total Entidade Skweak
B-numero_contrato     0     0                []
B-data_assinatura_contrato   0     0        []
B-processo_gdf   121     121        []


In [219]:
classes = ['B-numero_contrato',
 'B-processo_gdf',
 'I-processo_gdf',
 'B-data_assinatura_contrato',
 'I-numero_contrato',
 'I-data_assinatura_contrato']

In [220]:
len(labels_skweak_list)

21669

In [221]:
classes

['B-numero_contrato',
 'B-processo_gdf',
 'I-processo_gdf',
 'B-data_assinatura_contrato',
 'I-numero_contrato',
 'I-data_assinatura_contrato']

In [222]:
#Validação com sklearn
metrics.flat_f1_score(y_test, labels_skweak, average='weighted', labels=classes)

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.9814536151051818

In [223]:
#Validação com seqeval
from seqeval.metrics import f1_score
f1_score(y_test, labels_skweak, average='weighted')

0.35628298280362153

In [224]:
# classification_report Seqeval
import numpy as np
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

report = classification_report(y_test, labels_skweak, output_dict=False, mode='strict', scheme=IOB2)
print(report)
np.save("./Results/V1/METRICA_AVISO_LICITACAO_DIRETO_0.npy", report)

                                  precision    recall  f1-score   support

codigo_licitacao_sistema_compras       0.00      0.00      0.00        91
         data_abertura_licitacao       0.00      0.00      0.00       127
            modalidade_licitacao       0.99      0.82      0.90       139
                nome_responsavel       0.00      0.00      0.00       129
                numero_licitacao       0.99      0.87      0.93       143
                objeto_licitacao       0.00      0.00      0.00       111
                 orgao_licitante       0.00      0.00      0.00       101
                    processo_gdf       0.95      0.99      0.97       121
                 sistema_compras       0.00      0.00      0.00       158
                     tipo_objeto       0.00      0.00      0.00        31
      valor_estimado_contratacao       0.97      0.86      0.91        98

                       micro avg       0.98      0.35      0.52      1249
                       macro avg    

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [225]:
#classification_report sklearn
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
   gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=False)
print(report)

NameError: name 'gold_y_test' is not defined

In [ ]:
import numpy as np
report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    gold_y_test, labels_skweak, labels=classes, digits=3, output_dict=True)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_LB_DIRETO_V1_0.npy", report)